In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib as mpl
mpl.use('Agg')

mpl.rcParams['figure.figsize'] = (7,7)
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
mpl.rcParams.update({'font.size':22})

% matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [2]:
import halo_cnn
from halo_cnn.model import BaseHaloCNNRegressor

Using Theano backend.


In [3]:
m = BaseHaloCNNRegressor(input_shape=(48,1))

In [4]:
print(m)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 10)            60        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 5)             155       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 5)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 240)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               30848     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
__________

In [5]:
x = (2,3,5)
x[1:]

(3, 5)